# Useful Functions from Previous Classes

In [16]:
# Read substitution table from a table on a existing file.
# Assume the matrix is symmetric and first row contains the symbols in the alphabet
def read_submat_file(filename):
    f = open(filename, "r")
    alphabet = [symbol for symbol in f.readline().replace('\n', '').split('\t')]
        
    dic = {}
    for i, line in enumerate(f):
        line_symbol = line.replace('\n', '').split('\t')
        
        for j in range(0, len(line_symbol)):
            dic[alphabet[i] + alphabet[j]] = int(line_symbol[j])
            
    f.close()
    return dic

# Provides the score of a column alignment (between c1 and c2)
# Assume a constant gap penalty g and a substituin matrix sm
def score_alignment(c1, c2, sm, g):
    return g if c1 == '-' or c2 == '-' else sm[c1+c2]

def max3(v1, v2, v3):
    """Indicates which of the given integers is bigger: 1 2 or 3"""
    if v1 > v2:
        return 1 if v1 > v3 else 3
    else:
        return 2 if v2 > v3 else 3

# Smith-Waterman Alignment
def smith_waterman(seq1, seq2, sm, g):
    """Local Alignment using Smith-Waterman algorithm"""
    score = [[0]]
    trace = [[0]]
    maxscore = 0
    
    # initialize gaps in cols
    for i in range(1, len(seq1) + 1):
        score.append([0])
        trace.append([0])
    
    # initialize gaps in rows
    for j in range(1, len(seq2) + 1):
        score[0].append(0)
        trace[0].append(0)
        
    # apply the recurrence to fill the matrices
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            v1 = score[i-1][j-1] + score_alignment(seq1[i-1], seq2[j-1], sm, g)
            v2 = score[i-1][j] + g
            v3 = score[i][j-1] + g
            max_v = max(v1, v2, v3)
            
            score[i].append(0 if max_v <= 0 else max_v)
            trace[i].append(0 if max_v <= 0 else max3(v1, v2, v3))
            maxscore = maxscore if max_v < maxscore else max_v
    
    return (score, trace, maxscore)

# Coppied Teacher's code -> can it be improved?
def max_mat(mat):
    """finds the max cell in the matrix"""
    maxval = mat[0][0]
    maxrow = 0
    maxcol = 0
    for i in range(0,len(mat)):
        for j in range(0, len(mat[i])):
            if mat[i][j] > maxval:
                maxval = mat[i][j]
                maxrow = i
                maxcol = j
    return (maxrow, maxcol)

# Coppied Teacher's code -> can it be improved ???
def recover_align_local(score, trace, seq1, seq2):
    """recover one of the optimal alignments"""
    res = ["", ""]
    
    """determine the cell with max score"""
    i, j = max_mat(score)
    
    """terminates when finds a cell with zero"""
    while trace[i][j] > 0:
        if trace[i][j]==1:
            res[0] = seq1[i-1] + res[0]
            res[1] = seq2[j-1] + res[1]
            i -= 1
            j -= 1

        elif trace[i][j] == 3:
            res[0] = "-" + res[0]
            res[1] = seq2[j-1] + res[1]
            j -= 1

        elif trace[i][j] == 2:
            res[0] = seq1[i-1] + res[0]
            res[1] = "-" + res[1]
            i -= 1

    return res

# sw = smith_waterman('TACT', 'ACTA', read_submat_file('files/blosum62.mat'), -3)

# Sequence Similarity - 8/09

In [22]:
# Finds the most similar sequence to a query sequence
# The function should use the local alignment algorithm developed previously
# Input:
# * Query sequence (query)
# * List of sequence
# * Substitution Matrix
# * Gap penalty (g)
# * Return alignment with the best sequence
def seq_similarity(query, seqs, sm, g):
    sw_seqs = [smith_waterman(query, seq, sm, g) for seq in seqs]
    sw_max_seqs = [seq[2] for seq in sw_seqs]
    max_idx = sw_max_seqs.index(max(sw_max_seqs))
    score, trace, _ = sw_seqs[max_idx]
    
    return (score, recover_align_local(score, trace, query, seqs[max_idx]))

# Test
seq_similarity('ACTG', ['ATGC', 'ATTT', 'TACT', 'ATTGT'], read_submat_file('files/blosum62.mat'), 1)

([[0, 0, 0, 0, 0],
  [0, 1, 4, 5, 6],
  [0, 2, 5, 13, 14],
  [0, 5, 6, 14, 18],
  [0, 6, 7, 15, 19]],
 ['ACTG', 'ACT-'])

## BLAST

Programa desenvolvida para comprar não só sequências de nucleótidos, bem como sequências próteicas. Usado na comparação de sequências.